In [29]:
from sentinelhub import WebFeatureService, BBox, CRS, DataSource, SHConfig
import pandas as pd

In [19]:
INSTANCE_ID = '6247026c-d414-4d33-8457-a68a64247ba0'  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}}, 'coordinates': [[[[2.99981848315413, 19.89641809196557], [4.04871002343364, 19.893328548919364], [4.04237262477467, 18.90121793150136], [2.999819580310846, 18.904141724122724], [2.99981848315413, 19.89641809196557]]]]}, 'properties': {'id': 'S2A_OPER_MSI_L1C_TL_SGS__20151231T155113_A002736_T31QEB_N02.01', 'date': '2015-12-31', 'time': '10:30:09', 'path': 's3://sentinel-s2-l1c/tiles/31/Q/EB/2015/12/31/0', 'crs': 'EPSG:32631', 'mbr': '499980,2090220 609780,2200020', 'cloudCoverPercentage': 0}}
{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}}, 'coordinates': [[[[3.9553117480998132, 19.893854350479174], [5.00386045124805, 19.885138087588636], [4.99175814288768, 18.893466851037743], [3.9495385416715276, 18.90171552482112], [3.9553117480998132, 19.893854350479174]]]]}, 'pro

KeyboardInterrupt: 

In [22]:
search_bbox = BBox(bbox=[-5.27, 6.81, 5.5, 30], crs=CRS.WGS84)
search_time_interval = ('2015-06-01T00:00:00', '2015-12-31T23:59:59')


wfs_iterator = WebFeatureService(
    search_bbox,
    search_time_interval,
    data_source=DataSource.SENTINEL2_L1C,
    maxcc=.05,
    config=config
)

results = []
for tile_info in wfs_iterator:
    results.append(tile_info)

In [31]:
df = pd.DataFrame(wfs_iterator.get_tiles(), columns=['Tilename','Date','CloudCover'])

In [33]:
df2 = df.drop_duplicates(subset='Tilename')

In [34]:
df2

,Tilename,Date,CloudCover
0,31QEB,2015-12-31,0
1,31QFB,2015-12-31,0
2,31QFD,2015-12-31,0
3,31QFC,2015-12-31,0
4,31QFE,2015-12-31,0
...,...,...,...
2206,30QWD,2015-9-28,0
2214,30PVB,2015-9-28,0
2217,30PVC,2015-9-28,0
2495,30RVT,2015-9-14,0
